In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, InputLayer, Dropout, Dense, Flatten, Embedding
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

from tensorflow.keras.layers import concatenate
from sklearn.model_selection import train_test_split

In [38]:
df = pd.read_csv('data/affcon_final.csv')

In [7]:
df.columns

Index(['Input.convo_id', 'Input.train_test_val', 'Input.msg_id',
       'Input.timestamp', 'Input.full_text', 'affcon_gamemove',
       'affcon_reasoning', 'affcon_rapport', 'affcon_shareinformation',
       'Input.speaker', 'Input.reply_to', 'Input.speaker_intention',
       'Input.reciever_perception', 'Input.reciever',
       'Input.absolute_message_index', 'Input.relative_message_index',
       'Input.year', 'Input.game_score_speaker', 'Input.game_score_receiver',
       'Input.game_score_delta', 'Input.deception_quadrant', 'Input.num_words',
       'Input.num_characters', 'Input.sno', 'Input.sno1'],
      dtype='object')

In [14]:
df['Input.full_text'] = df['Input.full_text'].fillna(0)
df['Input.full_text'] = df['Input.full_text'].replace('\n','', regex=True)

In [43]:
train, test = train_test_split(df, test_size=0.2)

In [44]:
X_train = train['Input.full_text'].to_list()
y_train_rapport = train['affcon_rapport'].tolist()
y_train_share_information = train['affcon_shareinformation'].tolist()
y_train_reasoning = train['affcon_reasoning'].tolist()
y_train_gamemove = train['affcon_gamemove'].tolist()

X_test = test['Input.full_text'].to_list()
y_test_rapport = test['affcon_rapport'].tolist()
y_test_share_information = test['affcon_shareinformation'].tolist()
y_test_reasoning = test['affcon_reasoning'].tolist()
y_test_gamemove = test['affcon_gamemove'].tolist()

In [6]:
max_length = 100

from transformers import AutoTokenizer, TFAutoModel, AutoConfig, TFAutoModelForPreTraining 

model_name = 'bert-base-uncased'
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

auto_model = TFAutoModelForPreTraining.from_pretrained(model_name, config=config)

All model checkpoint layers were used when initializing TFBertForPreTraining.

All the layers of TFBertForPreTraining were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForPreTraining for predictions without further training.


In [47]:
Y_train_rapport_class = to_categorical(y_train_rapport)
Y_train_share_information_class = to_categorical(y_train_share_information)
Y_train_reasoning_class = to_categorical(y_train_reasoning)
Y_train_gamemove_class = to_categorical(y_train_gamemove)

Y_test_rapport_class = to_categorical(y_test_rapport)
Y_test_share_information_class = to_categorical(y_test_share_information)
Y_test_reasoning_class = to_categorical(y_test_reasoning)
Y_test_gamemove_class = to_categorical(y_test_gamemove)

In [48]:
X_train_text = tokenizer(
    text=X_train,
    add_special_tokens=True,
    max_length=max_length,
    padding='max_length',
    truncation=True,
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)

X_test_text = tokenizer(
    text=X_test,
    add_special_tokens=True,
    max_length=max_length,
    padding='max_length',
    truncation=True,
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)

In [8]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

Using TensorFlow backend.


In [53]:
input_ids_in = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')

embedding_layer = auto_model(input_ids_in)[0]
cls_token = embedding_layer[:,0,:]
X = tf.keras.layers.BatchNormalization()(cls_token)
X = tf.keras.layers.Dense(192, activation='relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)
X = tf.keras.layers.Dense(2, activation='softmax')(X)
gamemove_model = tf.keras.Model(inputs=input_ids_in, outputs = X)

gamemove_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m])
history = gamemove_model.fit(X_train_text['input_ids'], Y_train_gamemove_class, epochs=1, batch_size=16, 
                    validation_split=0.2, callbacks=[callback])
loss, accuracy, f1_score, precision, recall = gamemove_model.evaluate(X_test_text['input_ids'], Y_test_gamemove_class, verbose=0)
print(precision, recall, f1_score)

630/630 [==============================] - 245s 388ms/step - loss: 5.4234 - acc: 0.6438 - f1_m: 0.6438 - precision_m: 0.6438 - recall_m: 0.6438 - val_loss: 4.5926 - val_acc: 0.7006 - val_f1_m: 0.7011 - val_precision_m: 0.7011 - val_recall_m: 0.7011
0.7279040217399597 0.7279040217399597 0.7279039621353149


In [54]:
input_ids_in = tf.keras.layers.Input(shape=(128,), dtype='int32')

embedding_layer = auto_model(input_ids_in)[0]
cls_token = embedding_layer[:,0,:]
X = tf.keras.layers.BatchNormalization()(cls_token)
X = tf.keras.layers.Dense(192, activation='relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)
X = tf.keras.layers.Dense(2, activation='softmax')(X)
reasoning_model = tf.keras.Model(inputs=input_ids_in, outputs = X)

reasoning_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m])
history = reasoning_model.fit(X_train_text['input_ids'], Y_train_reasoning_class, epochs=1, batch_size=16, 
                    validation_split=0.2, callbacks=[callback])
loss, accuracy, f1_score, precision, recall = reasoning_model.evaluate(X_test_text['input_ids'], Y_train_reasoning_class, verbose=0)
print(precision, recall, f1_score)

630/630 [==============================] - 240s 380ms/step - loss: 3.8012 - acc: 0.7495 - f1_m: 0.7495 - precision_m: 0.7495 - recall_m: 0.7495 - val_loss: 3.6972 - val_acc: 0.7589 - val_f1_m: 0.7579 - val_precision_m: 0.7579 - val_recall_m: 0.7579


ValueError: Data cardinality is ambiguous:
  x sizes: 3148
  y sizes: 12590
Please provide data which shares the same first dimension.

In [55]:
# Other ideas: https://stackoverflow.com/questions/58900947/keras-tensorflow-merge-two-different-model-output-into-one

m = keras.layers.merge.Add([gamemove_model, reasoning_model])
out_m1 = M1(input_)
out_m2 = M2(out_m1)
out_m3 = M3(out_m1)
fout = m([out_m2, out_m3])
finalModel = Model(input="input_", output=fout)

AttributeError: module 'tensorflow.keras.layers' has no attribute 'Merge'

In [56]:
input_ids_in.shape

TensorShape([None, 128])

In [ ]:
from keras.layers import * 

commonInput = Input(shape=(1))

out1 = gamemove_model(commonInput)    
out2 = reasoning_model(commonInput)    
mergedOut = Add()([out1,out2])

#mergedOut = Add()([gamemove_model.output,reasoning_model.output])
mergedOut = Flatten()(mergedOut)    
mergedOut = Dense(256, activation='relu')(mergedOut)
mergedOut = Dropout(.5)(mergedOut)
mergedOut = Dense(128, activation='relu')(mergedOut)
mergedOut = Dropout(.35)(mergedOut)
mergedOut = Dense(2, activation='softmax')(mergedOut)  #Cuz binary

#mergedModel = Model([model1.input,model2.input], mergedOut)
mergedModel = Model(commonInput, mergedOut)
mergedModel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m])
mergedModel.fit(x=[X_train_text['input_ids'], Y_train_reasoning_class, epochs=1, batch_size=16, 
                    validation_split=0.2, callbacks=[callback])